### Imports

In [ ]:
from google.cloud import bigquery
import io
import json

### Big Query Client - Create Table

In [ ]:
bq_client = bigquery.Client()
project_id = '<<PROJECT_ID>>'
dataset_id = '<<DATASET_ID>>'
table_id = 'titanic_survived'
location = 'us-central1'

In [ ]:
schema = [
    bigquery.SchemaField("full_name", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("ssn", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("email", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("neighborhood", "STRING", mode="REQUIRED"),
]

fully_qualified_id = '{}'.format(project_id +'.'+ dataset_id +'.'+ table_id)
table = bigquery.Table(fully_qualified_id, schema=schema)
table = bq_client.create_table(table)  # Make an API request.
print(
    "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
)

In [ ]:
rows_to_insert = [
    {u"full_name": u"Pedro Sanchez", u"ssn": 12345678, u"email": u"pedrosanchez@google.com", u"neighborhood": u"Liniers"},
    {u"full_name": u"Pablo Perez", u"ssn": 87654321, u"email": "pabloperez@google.com", u"neighborhood": u"Mataderos"},
    {u"full_name": u"Georgina Azul", u"ssn": 65498721, u"email": "georginaazul@google.com", u"neighborhood": u"Palermo"},
    {u"full_name": u"Pepe Perez", u"ssn": 12387694, u"email": "pepeperez@google.com", u"neighborhood": u"Huechuraba"}
]

In [ ]:
errors = bq_client.insert_rows_json(fully_qualified_id, rows_to_insert)  # Make an API request.
if errors == []:
    print("New rows have been added.")
else:
    print("Encountered errors while inserting rows: {}".format(errors))

### Big Query Client - Get Table Schema

In [ ]:
dataset_ref = bq_client.dataset(dataset_id, project=project_id)
table_ref = dataset_ref.table(table_id)
table = bq_client.get_table(table_ref)
taxonomy_parent = "projects/<<PROJECT_ID>>/locations/us"

In [ ]:
f = io.StringIO("")
bq_client.schema_to_json(table.schema, f)
jeison = json.loads(f.getvalue())

In [ ]:
jeison

### Credentials

In [ ]:
from google.oauth2 import service_account
key_path = "./<<PROJECT_ID>>-<<KEY-FOR-CREDENTIAL>>.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

### Data Catalog - Taxonomy Creation

In [ ]:
#!pip install google-cloud-datacatalog
from google.cloud import datacatalog
from google.cloud.datacatalog_v1beta1 import PolicyTagManagerClient, types

In [ ]:
client = PolicyTagManagerClient(credentials = credentials)

In [ ]:
obj_taxonomy = types.Taxonomy()
obj_taxonomy.display_name = 'PII'

In [ ]:
taxonomy_response = client.create_taxonomy(parent=taxonomy_parent, taxonomy=obj_taxonomy)

### Data Catalog - Policy Tags Creation

In [ ]:
obj_PolicyTag_HIGH = types.PolicyTag()
obj_PolicyTag_HIGH.display_name= "HIGH"
policy_HIGH_response = client.create_policy_tag(parent=taxonomy_response.name, policy_tag=obj_PolicyTag_HIGH)

In [ ]:
obj_PolicyTag_MEDIUM = types.PolicyTag()
obj_PolicyTag_MEDIUM.display_name= "MEDIUM"
policy_MEDIUM_response = client.create_policy_tag(parent=taxonomy_response.name, policy_tag=obj_PolicyTag_MEDIUM)

In [ ]:
obj_PolicyTag_SSN = types.PolicyTag()
obj_PolicyTag_SSN.display_name= "SSN"
obj_PolicyTag_SSN.parent_policy_tag = policy_HIGH_response.name
policy_SSN_response = client.create_policy_tag(parent=taxonomy_response.name, policy_tag=obj_PolicyTag_SSN)

In [ ]:
obj_PolicyTag_EMAIL = types.PolicyTag()
obj_PolicyTag_EMAIL.display_name= "EMAIL"
obj_PolicyTag_EMAIL.parent_policy_tag = policy_MEDIUM_response.name
policy_EMAIL_response = client.create_policy_tag(parent=taxonomy_response.name, policy_tag=obj_PolicyTag_EMAIL)

### Data Catalog - Get Taxonomies and Policy Tags ID

In [ ]:
taxonomy_to_search = obj_taxonomy.display_name
policy_tag_to_search = 'Restricted'
column_to_attach_policy_tag = 'total'

In [ ]:
taxonomy_instance = ""
taxonomy_list = client.list_taxonomies(parent=taxonomy_parent)
for taxonomy in taxonomy_list:
    if (taxonomy.display_name == taxonomy_to_search):
        taxonomy_instance = taxonomy

if not taxonomy_instance:
    print("Taxonomy specified does not exist")
else:
    print(taxonomy_instance.name)

In [ ]:
policy_tag_instance = ""
policy_tags_list = client.list_policy_tags(parent=taxonomy_instance.name)
for policy_tag in policy_tags_list:
    if (policy_tag.display_name == policy_tag_to_search):
        policy_tag_instance = policy_tag

if not policy_tag_instance:
    print("Policy Tag specified does not exist")
else:
    print(policy_tag_instance.name)

### Big Query Schema Update

In [ ]:
policy_tag_instance.name

In [ ]:
from googleapiclient import discovery
bigquery_api = discovery.build('bigquery', 'v2', credentials=credentials)

In [ ]:
jeison

In [ ]:
for element in jeison:
    if element['name'] == column_to_attach_policy_tag:
        element['policyTags'] = {'names': [policy_tag_instance.name]}

In [ ]:
tables = bigquery_api.tables()

In [ ]:
jeison = {'schema': {'fields': jeison}}

In [ ]:
jeison

In [ ]:
tables.patch(projectId=project_id, datasetId=dataset_id, tableId=table_id, body=jeison).execute()

### IAM Policy

In [ ]:
policy_tag_instance.name

In [ ]:
iam_dc = {
   "resource":policy_tag_instance.name,
   "policy":{
      "bindings":[
         {
            "role":"roles/datacatalog.categoryFineGrainedReader",
            "members":[
               "user:example@gmail.com"
            ]
         }
      ]
   }
}

In [ ]:
client.set_iam_policy(iam_dc)

### Data Catalog - Create Tag Template

In [ ]:
from google.cloud import datacatalog_v1
tag_template = datacatalog_v1.types.TagTemplate()
datacatalog_client = datacatalog_v1.DataCatalogClient()

In [ ]:
tag_template.display_name = 'Demo Tag Template'

tag_template.fields['source'] = datacatalog_v1.types.TagTemplateField()
tag_template.fields['source'].display_name = 'Source of data asset'
tag_template.fields['source'].type_.primitive_type = datacatalog_v1.types.FieldType.PrimitiveType.STRING

tag_template.fields['num_rows'] = datacatalog_v1.types.TagTemplateField()
tag_template.fields['num_rows'].display_name = 'Number of rows in data asset'
tag_template.fields['num_rows'].type_.primitive_type = datacatalog_v1.types.FieldType.PrimitiveType.DOUBLE

tag_template.fields['has_pii'] = datacatalog_v1.types.TagTemplateField()
tag_template.fields['has_pii'].display_name = 'Has PII'
tag_template.fields['has_pii'].type_.primitive_type = datacatalog_v1.types.FieldType.PrimitiveType.BOOL

In [ ]:
expected_template_name = datacatalog_v1.DataCatalogClient.tag_template_path(project_id, location, 'example_tag_template')

In [ ]:
# Delete any pre-existing Template with the same name.
try:
    datacatalog_client.delete_tag_template(name=expected_template_name, force=True)
    print('Deleted template: {}'.format(expected_template_name))
except:
    print('Cannot delete template: {}'.format(expected_template_name))

# Create the Tag Template.
try:
    tag_template = datacatalog_client.create_tag_template(
        parent='projects/{}/locations/us-central1'.format(project_id),
        tag_template_id='example_tag_template',
        tag_template=tag_template)
    print('Created template: {}'.format(tag_template.name))
except OSError as e:
    print('Cannot create template: {}'.format(expected_template_name))
    print('{}'.format(e))

### Search Data Catalog Entry

In [ ]:
resource_name = '//bigquery.googleapis.com/projects/{}' \
                '/datasets/{}/tables/{}'.format(project_id, dataset_id, table_id)
table_entry = datacatalog_client.lookup_entry(request={"linked_resource": resource_name})

### Attach Tag to Data Catalog Entry

In [ ]:
tag = datacatalog_v1.types.Tag()

tag.template = tag_template.name
tag.name="my_tag"

tag.fields['source'] = datacatalog_v1.types.TagField()
tag.fields['source'].string_value = 'Hand-made example'

tag.fields['num_rows'] = datacatalog_v1.types.TagField()
tag.fields['num_rows'].double_value = 4

tag.fields['has_pii'] = datacatalog_v1.types.TagField()
tag.fields['has_pii'].bool_value = True

tag = datacatalog_client.create_tag(parent=table_entry.name, tag=tag)
print('Created tag: {}'.format(tag.name))

### Search Existent Tag Template

In [ ]:
scope = datacatalog.SearchCatalogRequest.Scope()
scope.include_project_ids.append(project_id)

In [ ]:
tag_template_name_search = 'example_tag_template'
name='projects/{}/locations/us-central1/tagTemplates/{}'.format(project_id,tag_template_name_search)

In [ ]:
search_tag_template = datacatalog_client.get_tag_template(name=name)

In [ ]:
search_tag_template

### Search Catalog

In [ ]:
search_result = datacatalog_client.search_catalog(scope=scope, query='tag:example_tag_template.has_pii=False')

In [ ]:
search_result

In [ ]:
search_result = datacatalog_client.search_catalog(scope=scope, query='customers')

In [ ]:
search_result